In [361]:
import pandas as pd
import numpy as np
import rpy2
import rpy2.robjects as ro
from rpy2 import robjects
from rpy2.robjects import r
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
import matplotlib.pyplot as plt
import networkx as nx
from heapq import merge

In [362]:
import sys
print(sys.version)
print(rpy2.__version__)
utils = importr('utils')
print(utils.packageVersion("rENA"))

3.6.8 (tags/v3.6.8:3c6b436a57, Dec 24 2018, 00:16:47) [MSC v.1916 64 bit (AMD64)]
2.9.5
[1] '0.1.6.1'



## Read Input as Pandas Dataframe

* Generate points rotade for each grupo of topicsID on dataFrame

In [363]:
pandas2ri.activate()
rena = importr('rENA')
posts = pd.read_excel('posts.xls', 'controle')

In [364]:
posts = posts[posts['phaseId'] != 0]

In [365]:
posts_topicId = []
for i in range(0,len(posts['topicId'].unique())):
    posts_topicId.append(posts[posts['topicId'] == posts['topicId'].unique()[i]])
#posts_topicId[1][['topicId','postParentId']].head()


##  ENA Set

In [366]:
pyset_list = []
for post in posts_topicId:
    accum =  rena.ena_accumulate_data( units = post[["postId"]], conversation = post[["postId"]], codes = post[["construction.intro",
       "scope.intro", "maintenance.intro", "general.comments",
       "construction.methods", "process.intro", "maintenance.management",
       "process.stages", "design.quality", "requirements.methods",
       "process.keyissues", "design.methods", "construction.languages",
       "requirements.intro", "design.intro"]],  weight_by = "identity", window_size_back = 1)
    pyset_list.append(rena.ena_make_set(enadata = accum))
len(pyset_list)

84

## Cosine Similarity

In [367]:
def get_distance_list(pyset_list):
    cos_distance_list = []
    for pyset in pyset_list:
        distance_topicID = []
        for i in range(0,len(np.array(pyset['points.rotated']))):
            distance_post = []
            for j in range(i, len(np.array(pyset['points.rotated']))):
                distance_post.append(1 - spatial.distance.cosine(np.array(pyset['points.rotated'])[i], np.array(pyset['points.rotated'])[j]))
            distance_topicID.append(distance_post)
        cos_distance_list.append(distance_topicID)
    return(cos_distance_list)
        
distance_list = get_distance_list(pyset_list)
        

## Implicit Relation

In [368]:
def get_implicit_edges(distance_list, limit):
    edge_list = []
    for similarity in distance_list:
        topic_edge = []
        for i in range(0,len(similarity)):
            for j in range(0,len(similarity[i])):
                if (abs(similarity[i][j]) >= limit):
                    topic_edge.append((j+i, i))
        edge_list.append(topic_edge)
    return edge_list
    
implicit_edges = get_implicit_edges(distance_list, 0.90)


## Explicit Relation

In [369]:
def get_explicit_edges(postList):
    explicit_edges_lst = []
    for forum in postList:
        postParentId = forum['postParentId'].tolist()
        postId = forum['postId'].tolist()
        edge_list = []
        for i in range(0,len(postParentId)):
            if postParentId[i] in postId:
                edge_list.append((i, postId.index(postParentId[i])))
        explicit_edges_lst.append(edge_list)
    return explicit_edges_lst
        
explicit_edges = get_explicit_edges(posts_topicId)
#print(postParentId)
#print(postId)
#print(edge_list)
#postId.index(postParentId[2])
#explicit_edges[2]



## Identificar quem é o lider e o participante nos nós. (atribuir cores quando desenhar)
#### Participante Verde, Líder Vermelho, inicial nó quadrado

In [370]:
def get_roleList(posts_topicId):
    liderLst = []
    participanteLst = []
    for forum in posts_topicId:
        roleList = ((forum['UserRole (11- expert/part1, 12 - expert/part2,   21- participant/part1, 22- participant/part2)'] == 11) | 
                    (forum['UserRole (11- expert/part1, 12 - expert/part2,   21- participant/part1, 22- participant/part2)'] == 12)).tolist()
        lider = []
        participante = []
        for i in range(0,len(roleList)):
            if roleList[i] == True:
                lider.append(i)
            else:
                participante.append(i)

        liderLst.append(lider)
        participanteLst.append(participante)
    return (liderLst,participanteLst)

liderLst, participanteLst = get_roleList(posts_topicId)
        

## Clean Loops in Implicit Edges

In [371]:
def removeLoops(implicit_edges):
    newImplicitEdges = []
    for edgeList in implicit_edges:
        newEdges = []
        for i in range(0,len(edgeList)):
            if edgeList[i][0] != edgeList[i][1]:
                newEdges.append(edgeList[i])
        newImplicitEdges.append(newEdges)
                
    return (newImplicitEdges)

def removeEdges2Liders(implicit_edges, liderLst):
    newImplicitEdges = []
    for i in range(0,len(implicit_edges)):
        newEdges = []
        for j in range(0,len(implicit_edges[i])):
            if ((implicit_edges[i][j][0] == 0)  and (implicit_edges[i][j][1] in liderLst[i])):
#            if ((implicit_edges[i][j][0] in liderLst[i])  and (implicit_edges[i][j][1] in liderLst[i])):
                continue
            else:
                newEdges.append(implicit_edges[i][j])
        newImplicitEdges.append(newEdges)
                
    return (newImplicitEdges)

def addImpEdgesRevers(implicit_edges):
    
    for i in range(0,len(implicit_edges)):
        reversed_edges = [edges[::-1] for edges in implicit_edges[i]]
        implicit_edges[i].extend(reversed_edges)
    return (implicit_edges)
        
    
implicit_edges = removeLoops(implicit_edges)
implicit_edges = removeEdges2Liders(implicit_edges, liderLst)
#implicit_edges = addImpEdgesRevers(implicit_edges)


### Create Hierarchical Graph

In [372]:
#print(liderLst[0])
#print(participanteLst[0])
#print('--')
#print(implicit_edges[0])
#print('--')
#print(graphLst[0].edges())
#print('--')
#print(nx.is_connected(nx.Graph(graphLst[0].edges())))


In [373]:
def getCreateGraphs(posts_topicId,explicit_edges,implicit_edges,liderLst,participanteLst):
    graphLst = []
    for i in range(0,len(posts_topicId)):
        G = nx.DiGraph()
        G.add_nodes_from(list(merge(liderLst[i],participanteLst[i])))
        G.add_edges_from(explicit_edges[i])
        G.add_edges_from(implicit_edges[i])
        graphLst.append(G)
    return(graphLst)

graphLst = getCreateGraphs(posts_topicId,explicit_edges,implicit_edges,liderLst,participanteLst)

### Contar as ligações do nós líderes e participantes.

In [374]:
def get_nodes_connections(graphLst,liderLst,participanteLst):
    liderConLst = []
    participantConLst = []
    for i in range(0,len(graphLst)):
        degree = graphLst[i].out_degree()
        liderCon = []
        participantCon = []
        for node in liderLst[i]:
            liderCon.append(degree[node])
        for node in participanteLst[i]:
            participantCon.append(degree[node])
        liderConLst.append(liderCon)
        participantConLst.append(participantCon)
    return(liderConLst,participantConLst)

liderConLst,participantCon  = get_nodes_connections(graphLst,liderLst,participanteLst)
        

## Classificar forum em Instructor-Centered ou Synergistic

In [375]:
def get_forum_class(posts_topicId,liderConLst,participantCon):
    forumClass = []
    for i in range(0,len(posts_topicId)):
        if (sum(liderConLst[i]) > sum(participantCon[i])):
            forumClass.append('Instructor-Centered')
        else:
            if (sum(liderConLst[i]) < sum(participantCon[i])):
                forumClass.append('Synergistic')
            else:
                forumClass.append('Developing-Synergism')
                
    return(forumClass)
forumClass = get_forum_class(posts_topicId,liderConLst,participantCon)          
#forumClass
# graphLst[1].degree()
# labels = dict(zip(range(0,len(posts_topicId[1]['postUserId'].tolist())),posts_topicId[1]['postUserId'].tolist()))
# degree = graphLst[1].degree()
# #participanteLst[1]
#print(sum(liderConLst[1]),sum(participantCon[1]))

## Classificar forum como Scattered

In [376]:
def get_disconnectedGraphs(graphLst):
    scattered =[]
    for graph in graphLst:
        G = nx.Graph()
        G.add_edges_from(graph.edges())
        scattered.append(nx.is_connected(G))
    return (scattered)
def get_forum_scattered(forumClass,scattered):
    for i in range(0,len(scattered)):
        if (scattered[i]== False):
            forumClass[i] = 'Scattered'
    return (forumClass)
scattered = get_disconnectedGraphs(graphLst)
forumClass = get_forum_scattered(forumClass,scattered)

## Obter porcentagens do phaseId e gera planilha

In [377]:
def getGraphInfo(liderConLst,participantCon):
    liderDegreeSum = []
    participantDegreeSum =[]
    liderLen = []
    participantLen = []
    
    for degreeList in liderConLst:
        liderDegreeSum.append(sum(degreeList))
        liderLen.append(len(degreeList))
    for degreeList in participantCon:
        participantDegreeSum.append(sum(degreeList))
        participantLen.append(len(degreeList))
    return (liderDegreeSum,participantDegreeSum,liderLen,participantLen)

liderDegreeSum,participantDegreeSum,liderLen,participantLen = getGraphInfo(liderConLst,participantCon)
            

In [378]:
def getFreqLstDf(posts_topicId):
    df = pd.DataFrame(columns=[0,1,2,3,4])
    for i in range(0,len(posts_topicId)):
        #freq = posts_topicId[i]['phaseId'].value_counts(normalize=True) * 100
        freq = posts_topicId[i]['phaseId'].value_counts()
        df.loc[i] = freq  
    return (df)


freq_df = getFreqLstDf(posts_topicId)
freq_df['Class'] = forumClass
freq_df['topicId']= posts['topicId'].unique()
freq_df['liderDegreeSum'] = liderDegreeSum
freq_df['participantDegreeSum'] = participantDegreeSum
freq_df['liderLen'] = liderLen
freq_df['participantLen'] = participantLen

freq_df.to_csv("final3_090.csv")

#freq_df[[0,1,2,3,4]] = freq_df[[0,1,2,3,4]].apply(pd.to_numeric)
#freq_df.head()
#posts_topicId[0]['phaseId'].value_counts()

In [379]:
#freq_df[freq_df[4].isna()]
#graphLst[0].in_degree()

## STOP HERE

## Gerar imagens dos grafos de todos os Fórums

In [380]:
def genGraphImages(posts_topicId, graphLst, implicit_edges, explicit_edges, liderLst, participanteLst):
    
    for i in range(0,len(posts_topicId)):
        print(i)
        labels = dict(zip(range(0,len(posts_topicId[i]['postUserId'].tolist())),posts_topicId[i]['postId'].tolist()))
        graphLst[i].remove_edges_from(implicit_edges[i])
        pos=nx.fruchterman_reingold_layout(graphLst[i])
        plt.figure(3,figsize=(10,10)) 


        nx.draw_networkx_nodes(graphLst[i], pos, nodelist=liderLst[i],
                               node_color='r', node_size=700, alpha=0.2, with_labels=True)

        nx.draw_networkx_nodes(graphLst[i], pos, nodelist=participanteLst[i],
                               node_color='g', node_size=700, alpha=0.2, with_labels=True)
        if 0 in liderLst[i]:
            nx.draw_networkx_nodes(graphLst[i], pos, nodelist=[0],
                               node_shape='s',node_color='r', node_size=900, alpha=0.2, with_labels=True)
        else:
            nx.draw_networkx_nodes(graphLst[i], pos, nodelist=[0],
                               node_shape='s',node_color='g', node_size=900, alpha=0.2, with_labels=True)

        # # edges
        nx.draw_networkx_edges(graphLst[i], pos, edgelist=explicit_edges[i],
                                width=1, edge_color = 'b')
        nx.draw_networkx_edges(graphLst[i], pos, edgelist=implicit_edges[i],style='dashed',
                                alpha=0.5,  arrows=True, edge_color='y')
        # # labels
        nx.draw_networkx_labels(graphLst[i], pos, labels, font_size=12, font_family='sans-serif')
        plt.legend()
        plt.axis('off')
        plt.savefig('output/'+ "final3_090_" + str(i)+ '.png', format="PNG")
        plt.clf()


genGraphImages(posts_topicId, graphLst, implicit_edges, explicit_edges, liderLst, participanteLst)
    

No handles with labels found to put in legend.


0


No handles with labels found to put in legend.


1


No handles with labels found to put in legend.


...

No handles with labels found to put in legend.


82


No handles with labels found to put in legend.


83


<Figure size 720x720 with 0 Axes>